<a href="https://colab.research.google.com/github/ShagufthaZK/IR_project/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flask-ngrok

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/quizgen')
def quizgen():
    return render_template('quizgen.html')